# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import warnings

# Import API key
from api_keys import geoapify_key
warnings.filterwarnings("ignore")

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Bootcamp/moduleAPIChallenge/Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Bootcamp/moduleAPIChallenge/Starter_Code/output_data/cities.csv'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    color='City', 
    size='Humidity',
    scale= 0.8,
    title='City Map with Humidity',
    xlabel='Longitude', 
    ylabel='Latitude',
    hover_cols=['City', 'Humidity'],
    geo = True, tiles="OSM")

# Display the map
map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df['Max Temp'] >= 21) &  #Max temperature greater than or equal to 21 degrees Celsius
    (city_data_df['Max Temp'] <= 27) &  #Max temperature less than or equal to 27 degrees Celsius
    (city_data_df['Humidity'] <= 100) &  #Humidity less than or equal to 100%
    (city_data_df['Wind Speed'] < 4.50) & #Wind Speed less than 4.50
    (city_data_df['Cloudiness'] == 0))  #Example: Cloudiness equals 0%

#Create new DataFrame
ideal_city_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Organize the DataFrame by the 'City_ID' column in ascending order
ideal_city_df = ideal_city_df.sort_values(by='City_ID')

# Display sample data
ideal_city_df

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name='')

# Display sample data
hotel_df

In [ ]:
#Loop through the DataFrame Lat and Lng to view the JSON response for the next step.
##NOTE: A lot of this will be repeated in the next step, just to show the format
##in the way it is required for this assigment. I need to view the JSON object to extract the 'keys'
for index, row in hotel_df.iterrows():
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Set parameters to search for a hotel
    radius = 10000
    latitude = row['Lat']
    longitude = row['Lng']
    categories = "accommodation.hotel"
    limit = 1
    filter_geometry = f'circle:{longitude},{latitude},{radius}'
    bias = f'proximity:{longitude:.6f},{latitude:.6f}'  
    params = {
        'categories': categories,
        'limit': limit,
        'filter[geometry]': filter_geometry,
        'bias': bias,
        'apiKey': geoapify_key}  

    # Make an API request using the query URL
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Print the JSON response for each row
    print(f"Hotel information for {row['City']} - {row['Country']}:")
    print(json.dumps(name_address, indent=4, sort_keys=True))
    print("\n")

In [ ]:
#Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
# Set the radius to 10,000 meters (10 kilometers)
radius = 10000 
params = {
        'categories': categories,
        'limit': limit,
        'filter[geometry]': filter_geometry,
        'bias': bias,
        'apiKey': geoapify_key}  

categories = "accommodation.hotel"
limit = 1
filter_geometry = f'circle:{longitude},{latitude},{radius}'
bias = f'proximity:{longitude},{latitude}' 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
        hotel_name = name_address["features"][0]["properties"]["address_line1"]
        # Update the 'Hotel_Name' column in the DataFrame
        hotel_df.at[index, "Hotel_Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel_Name']]

In [ ]:
#Step 5: Add the hotel name and the country as additional information 
#         in the hover message for each city in the map.

%%capture --no-display

# Configure the map plot
map_plot2 = ideal_city_df.hvplot.points('Lng', 'Lat', color='City',
                                        size='Humidity',
                                        frame_width=700,
                                        frame_height=500,
                                        title='Ideal Vacation Spot',  
                                        geo=True, 
                                        tiles="OSM", 
                                        hover_cols=['City', 'Country', 'Humidity', 'Hotel_Name'])

# Display the map
map_plot2